In [1]:
import pandas as pd
import numpy as np

from os import walk
from functools import reduce


POWIATY_PATH = 'data/Powiat/'
SUBREGIONS_PATH = 'data/Subregion/'
REGIONS_PATH = 'data/Region/'

DF_UNITS = 'data/jednostki.csv'
DF_MAIN = 'data/df_main.csv'
DF_MAIN_MOD = 'data/df_main_mod.csv'
DF_MAIN_MOD_NORETIRES = 'data/df_main_mod_noretires.csv'

pd.set_option('display.max_columns', 60)
pd.set_option('display.max_rows', 160)

# Utworzenie głównej tabeli
Oraz dołączenie do niej danych p2013-motele-kategorii-sss według roku i powiatu. Zakres dat tej zmiennej jest największy spośród zmiennych (1995-2021).</br>
Dzięki temu kolumnę year bezproblemowo uczynimy kolejną składową klucza złożonego, zaraz po ID_region, ID_subregion oraz ID_powiatu.

## Przygotowanie klucza ID_region-ID_subregion-ID_powiatu
Kasowane są tabele pośrednie a całość zapisana zostaje w data/region-subregion-powiat.csv

In [ ]:
df_units = pd.read_csv(DF_UNITS)
df_units.drop(['kind', 'name'], axis=1, inplace=True)
df_units = df_units.drop(0)
df_units = df_units.astype({'ID_unitPatent': 'int64'})

df_subregions = df_units[df_units.level == 4].copy()
df_subregions.drop(['level'], axis=1, inplace=True)
df_subregions.rename(columns={'ID_unitPatent' : 'ID_region',
                              'ID_unit'       : 'ID_subregion'}, inplace = True)

df_powiaty = df_units[df_units.level == 5].copy()
df_powiaty.drop(['level'], axis=1, inplace=True)
df_powiaty.rename(columns={'ID_unitPatent' : 'ID_subregion',
                           'ID_unit'       : 'ID_powiatu'}, inplace = True)
df_powiaty = df_powiaty.astype({'ID_subregion': 'int64'})

df_region_powiat = pd.merge(df_subregions, df_powiaty, on=['ID_subregion'], how='inner')

del df_subregions
del df_powiaty

In [ ]:
df_region_powiat.to_csv(f'data/region-subregion-powiat.csv')

In [ ]:
df_region_powiat

In [ ]:
df_region_powiat = pd.read_csv('data/region-subregion-powiat.csv', index_col=[0])
df_motels_sss = pd.read_csv('data/Powiat/P2013-motele-kategorii-sss.csv', index_col=[0])
df_main = df_motels_sss.merge(df_region_powiat, on='ID_powiatu').reset_index().set_index(['ID_region', 'ID_subregion', 'ID_powiatu', 'year'])
df_main = df_main.reset_index().set_index(['index'])

## Zebranie danych

### Zebranie danych z poziomu regionu
Mergujemy listę regionalnych w jedną za pomocą reduce, dając jako pierwsze w naszej liście df_main. </br>
Dokonujemy merga na kluczach "ID_region" oraz "year".

In [ ]:
dfs_regio = [pd.read_csv(f'{REGIONS_PATH}{filename}').copy() 
             for filename in next(walk(REGIONS_PATH), (None, None, []))[2]]

dfs_regio.insert(0, df_main.reset_index().set_index(['index']).copy())

df_main = reduce(lambda left, right:
                 pd.merge(left, right, on=['ID_region', 'year'], how='left'),
                 dfs_regio)

del dfs_regio

In [ ]:
df_main

### Zebranie danych z poziomu podregionu

Metoda pozostaje niezmienna. Tym razem tworzymy listę zmiennych na poziomie podregionu i ją przy pomocy reduce mergujemy df_main "na lewo" z elementami listy.<br>
Merga dokonujemy na kluczach "ID_subregion" oraz "year".


In [ ]:
dfs_subregio = [pd.read_csv(f'{SUBREGIONS_PATH}{filename}').copy() 
             for filename in next(walk(SUBREGIONS_PATH), (None, None, []))[2]]

dfs_subregio.insert(0, df_main.copy())

df_main = reduce(lambda left, right:
                pd.merge(left, right, on=['ID_subregion', 'year'], how='left'),
            dfs_subregio)

del dfs_subregio

In [ ]:
df_main

### Zebranie danych z poziomu powiatu
Merga dokonujemy na kluczach "ID_powiatu" oraz "year". Warto nadmienić, że cel przewidywań - p3350-stopień-wykorzystania-miejsc-noclegowych - importujemy odrazu jako ostatnią kolumnę.


In [ ]:
dfs_powiat = [pd.read_csv(f'{POWIATY_PATH}{filename}').copy() 
             for filename in next(walk(POWIATY_PATH), (None, None, []))[2]]

dfs_powiat.insert(0, df_main.copy())

df_main = reduce(lambda left, right:
                pd.merge(left, right, on=['ID_powiatu', 'year'], how='left'),
            dfs_powiat)

del dfs_powiat

Powinniśmy skasować powstałą teraz kolumnę 'p2013-motele-kategorii-sss_x', a tę z suffixem '_y' z niego ogołocić.

In [ ]:
df_main.drop(columns='p2013-motele-kategorii-sss_x', inplace=True)
df_main.rename(columns={'p2013-motele-kategorii-sss_y' : 'p2013-motele-kategorii-sss'}, inplace=True)

In [ ]:
df_main.to_csv(DF_MAIN)

In [ ]:
df_main

# Modyfikacja głównej tabeli
Dokonamy modyfikacji w celu uniknięcia lookaheadów i uzupełnienia danych. Będziemy działać na kopii 'df_main_mod' utworzonej przed chwilą tabeli.

In [9]:
df_main_mod = pd.read_csv(DF_MAIN, index_col=[1,2,3,4])
df_main_mod.drop(df_main_mod.columns[0], axis=1, inplace=True)

## Usuwanie danych
Chcąc przewidzieć procent obłożenia miejsc noclegowych nie możemy używać danych sumarycznych z bieżącego roku. Wszystkie inne dane zbierane są więc na pewno przed nimi, wykluczając lookahead.<br>
Przesuwamy zatem każdą wartość z docelowej kolumny o rekord w tył (np. wartość z 2019 to teraz wartość z indeksem 2018), dodając do jej nazwy człon "-w-kolejnym-roku".

Wiąże się to też z wykluczeniem najnowszego rocznika z danych treningowych.

In [10]:
for powiat in df_main_mod.index.unique(level=2):
    df_main_mod.update(df_main_mod.xs(powiat, level=2)['p3350-stopień-wykorzystania-miejsc-noclegowych'].shift(-1))
    
df_main_mod.drop(index=2020, level=3, inplace=True)
df_main_mod.rename({'p3350-stopień-wykorzystania-miejsc-noclegowych' : 'p3350-stopień-wykorzystania-miejsc-noclegowych-w-kolejnym-roku'}, axis=1, inplace=True)

### Usunięcie rekordów bez danych szukanych
Podejmując się tego w tym momencie zminimalizujemy rozmiary tabeli, czyniąc kolejne jej modyfikacje mniej obciążającymi obliczeniowo - poddawalibyśmy modyfikacjom rekordy od początku spisane na skasowanie.<br>
Okazuje się, że kasujemy ich ponad połowę. Dodajemy do tego rekordy gdzie wynosi to zero, które z dużą pewnością stanowią wyłącznie anomalię.

In [11]:
df_main_mod.replace({'p3350-stopień-wykorzystania-miejsc-noclegowych-w-kolejnym-roku' : 0}, value = np.nan, inplace=True)
df_main_mod.dropna(subset=['p3350-stopień-wykorzystania-miejsc-noclegowych-w-kolejnym-roku'], inplace=True)

### Usuwanie rekordów bez żadnych danych poza wynikiem
Okazuje się jednak, że takowych nie ma.

In [12]:
df_main_mod[df_main_mod.drop(columns='p3350-stopień-wykorzystania-miejsc-noclegowych-w-kolejnym-roku').isna().all(axis=1)]

,,,,p1767-przyjazdy,p1767-tranzyt,p1767-udział-ruchu-pasażerów-polska-100,p1767-wyjazdy,p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności,p2424-linie-kolejowe-ogółem-na-100-km2,p2504-1-kwartał-ogółem,p2504-2-kwartał-ogółem,p2504-3-kwartał-ogółem,p2504-4-kwartał-ogółem,p2861-przeciętna-liczba-emerytów-i-rencistów-ogółem,p3441-liczba-nowo-utworzonych-miejsc-pracy,p3441-liczba-zlikwidowanych-miejsc-pracy,p3571-dalekobieżne,p3571-miejskie,p3571-podmiejskie,p3571-regionalne,p3603-przewozy-pasażerskie-na-1-mieszkańca,p3603-przewozy-pasażerów-w-mln,p3501-produkt-krajowy-brutto-na-1-mieszkańca,p2013-hotele-4-i-5-gwiazdkowemotelepensjonaty-w-ogólnej-liczbie-hotelimotelipensjonatów-ogółem,p2013-hotele-kategorii-s,p2013-hotele-kategorii-ss,p2013-hotele-kategorii-sss,p2013-hotele-kategorii-ssss,p2013-hotele-kategorii-sssss,p2013-motele-kategorii-s,p2013-motele-kategorii-ss,p2013-motele-kategorii-sss,p2013-motele-kategorii-ssss,p2013-motele-kategorii-sssss,p2013-pensjonaty-kategorii-s,p2013-pensjonaty-kategorii-ss,p2013-pensjonaty-kategorii-sss,p2013-pensjonaty-kategorii-ssss,p2013-pensjonaty-kategorii-sssss,p2420-pojazdy-samochodowe-i-ciągniki-na-1000-ludności,p2420-samochody-osobowe-na-1000-ludności,p2497-ogółem,p2497-przeciętne-miesięczne-wynagrodzenia-brutto-w-relacji-do-średniej-krajowej-polska100,p2596-lasy-gminne-powierzchnia,p2596-parki-spacerowo-wypoczynkowe-obiekty,p2596-parki-zieleńce-i-tereny-zieleni-osiedlowej-powierzchnia,p2596-zieleń-uliczna-powierzchnia,p2596-zieleńce-obiekty,p2617-krajowy-przyjazdy,p2617-krajowy-wyjazdy,p2617-międzynarodowy-przyjazdy,p2617-międzynarodowy-wyjazdy,p2670-ogółem,p2759-turyści-zagraniczni-nierezydenci-ogółem,p2759-turyści-zagraniczni-nierezydenci-pozostałe-kraje,p2813-handel-naprawa-pojazdów-samochodowych-transport-i-gospodarka-magazynowa-zakwaterowanie-i-gastronomia-informacja-i-komunikacja-ogółem,p3303-udział-parków-zieleńców-i-terenów-zieleni-osiedlowej-w-powierzchni-ogółem,p3350-stopień-wykorzystania-miejsc-noclegowych-w-kolejnym-roku
ID_region,ID_subregion,ID_powiatu,year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [13]:
df_main_mod.to_csv(DF_MAIN_MOD)

## Uzupełnianie danych

In [14]:
df_main_mod = pd.read_csv(DF_MAIN_MOD)

### Odnajdywanie danych do uzupełnienia

Nie są nimi wyłącznie wartości nieokreślone typu nan, ale również często pozbawione sensu wartości określone. Są nimi zera w:

1. p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności i linie-kolejowe-ogółem-na-100-km2
2. p2504 i kwartalne pkb
3. p2861-przeciętna-liczba-emerytów-i-rencistów-ogółem
4. p3571 i przewozy dalekobieżne, miejskie, podmiejskie oraz regionalne
5. p3603 i przewozy-pasażerskie-na-1-mieszkańca oraz przewozy-pasażerów-w-mln
6. p1767-udział-ruchu-pasażerów-polska-100 podczas gdy te same rekordy mają niezerowe wartości w p1767-wyjazdy czy p1767-przyjazdy 

Zdaje się, że najwięcej braków w danych występuje w regionach o numerach 71410000000 i 71420000000, czyli odpowiednio regionie warszawskim stołecznym i regionie mazowieckim regionalnym. Daje nam to wybór między usunięciem wszystkich rekordów o tym indeksie lub uzupełnienie ich. Pierwsze będzie nadal możliwe po wykonaniu tego drugiego, więc to je obieramy.

In [15]:
df_main_mod.replace({'p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności' : 0, 
                     'p2424-linie-kolejowe-ogółem-na-100-km2' : 0,
                     'p1767-tranzyt' : 0,
                     'p1767-przyjazdy' : 0,
                     'p1767-wyjazdy' : 0,
                     'p1767-udział-ruchu-pasażerów-polska-100' : 0,
                     'p2504-1-kwartał-ogółem': 0,
                     'p2504-2-kwartał-ogółem': 0,
                     'p2504-3-kwartał-ogółem': 0,
                     'p2504-4-kwartał-ogółem': 0,
                     'p2861-przeciętna-liczba-emerytów-i-rencistów-ogółem' : 0,
                     'p3501-produkt-krajowy-brutto-na-1-mieszkańca': 0,
                     'p3603-przewozy-pasażerów-w-mln': 0,
                     'p3603-przewozy-pasażerskie-na-1-mieszkańca': 0,
                     'p2497-ogółem' : 0}, value = np.nan, inplace=True)
df_groupby_region_temp = df_main_mod.drop(labels='ID_region', axis=1).isna().groupby(df_main_mod.ID_region).sum()

In [16]:
df_groupby_region_temp.loc[:,(df_groupby_region_temp != 0).any()]

,p1767-przyjazdy,p1767-tranzyt,p1767-udział-ruchu-pasażerów-polska-100,p1767-wyjazdy,p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności,p2424-linie-kolejowe-ogółem-na-100-km2,p2504-1-kwartał-ogółem,p2504-2-kwartał-ogółem,p2504-3-kwartał-ogółem,p2504-4-kwartał-ogółem,p2861-przeciętna-liczba-emerytów-i-rencistów-ogółem,p3571-dalekobieżne,p3571-miejskie,p3571-podmiejskie,p3571-regionalne,p3603-przewozy-pasażerskie-na-1-mieszkańca,p3603-przewozy-pasażerów-w-mln
ID_region,,,,,,,,,,,,,,,,,
11210000000,0,0,0,0,0,0,0,0,0,0,0,82,82,82,82,0,0
12410000000,0,0,0,0,0,0,0,0,0,0,0,130,130,130,130,0,0
20810000000,0,126,84,0,0,0,0,0,0,0,0,56,56,56,56,0,0
23010000000,0,0,0,0,0,0,0,0,0,0,0,140,140,140,140,0,0
23210000000,0,145,0,0,0,0,0,0,0,0,0,78,78,78,78,0,0
30210000000,0,0,0,0,0,0,0,0,0,0,0,110,110,110,110,0,0
31610000000,118,118,118,118,0,0,0,0,0,0,0,46,46,46,46,0,0
40410000000,0,0,0,0,0,0,0,0,0,0,0,92,92,92,92,0,0
42210000000,0,20,0,0,0,0,0,0,0,0,0,79,79,79,79,0,0


In [17]:
del df_groupby_region_temp

### 1. Linie połączeń kolejowych (p2424)

Mało prawdopodobnym jest, że ich liczba to okrągłe zero w danym regionie, a na dodatek wszystkie wpisy na lata 2009-2019 dla wspomnianych regionów to zera. Jednakże usunięty przedtem rekord z 2020 zawiera dane z tej zmiennej. Te powtórzymy dla tych regionów w każdym z pozostałych pól kolumny.

In [18]:
pd.read_csv(DF_MAIN, index_col=[1,4]).loc[[(71410000000, 2020), (71420000000, 2020)], ['p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności', 'p2424-linie-kolejowe-ogółem-na-100-km2']]

p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności  \
ID_region   year                                                       
71410000000 2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
            2020                                                2.4    
71420000000 2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    
            2020                                                4.2    

                  p2424-linie-kolejowe-ogółem-na-100-km2  
ID_region   year                                          
71410000000 2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
            2020                                    12.2  
71420000000 2020                                     3.3  
            2020                                     3.3  
            2020       

In [19]:
df_main_mod.set_index(['ID_region', 'ID_subregion', 'ID_powiatu', 'year'], inplace=True)

In [20]:
df_main_mod.loc[[(71410000000, 71413000000, 71413032000, 2019),
                 (71420000000, 71427300000, 71427338000, 2019)],
                ['p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności',
                 'p2424-linie-kolejowe-ogółem-na-100-km2']] = [[2.4, 12.2],[4.2, 2.3]]

In [21]:
df_main_mod.loc[:, ['p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności', 'p2424-linie-kolejowe-ogółem-na-100-km2']] = \
df_main_mod.loc[:, ['p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności', 'p2424-linie-kolejowe-ogółem-na-100-km2']].bfill()

### 2. Kwartalne PKB (p2504)

Aby uzyskać lepsze oszacowanie, dane powinny być zależne od regionu, dla którego w konkretnych latach brakuje kwartalnego PKB. (Uzupełnienie tej wartości średnią z całej kolumny byłoby znacznie mniej informatywne).<br>
Wyliczymy średnią dla konkretnego regionu i nią uzupełnimy braki.

In [22]:
df_main_mod.replace({'p2504-1-kwartał-ogółem' : 0,
                     'p2504-2-kwartał-ogółem' : 0,
                     'p2504-3-kwartał-ogółem' : 0,
                     'p2504-4-kwartał-ogółem' : 0}, value = np.nan, inplace=True)

In [23]:
df_main_mod.reset_index(inplace=True)
df_p2504_means = df_main_mod.groupby(['ID_region']).mean()[['p2504-1-kwartał-ogółem',
                                                            'p2504-2-kwartał-ogółem',
                                                            'p2504-3-kwartał-ogółem',
                                                            'p2504-4-kwartał-ogółem']]
df_main_mod = df_main_mod.set_index(['ID_region']).fillna(df_p2504_means)

del df_p2504_means

In [ ]:
df_main_mod.to_csv(DF_MAIN_MOD)

### 3. Liczby emerytów (p2861)

Problem stanowi fakt, iż dla wspomnianych wcześniej dwóch regionów nie posiadamy żadnych danych. Występują one jednak dla poziomu agregującego je oba w województwo mazowieckie. Pozostawia to następujące podejścia:
- Wyrzucenie kolumny z emerytami.
- Wyrzucenie rekordów o regionie warszawskim i mazowieckim.
- Uzupełnienie danymi zagregowanymi z poziomu województwa (możliwe jest zagregowanie obu regionów w jeden - bardzo czasochłonne zważywszy na pozostałe zmienne).
- Uzupełnienie danymi dostępnymi dla innych regionów.

Zastosowane trzeciego z nich w tym przypadku byłpby następujące: skorzystamy z danch o emerytach zagregowanych dla województwa i podzielimy je między regiony względem proporcji populacji obu z nich. "Byłoby", gdyby nie fakt, że w momencie pisania tej części kodu serwery gus'u przestały odpowiadać na zapytania ich własnego api, a przykłady użycia api z ich witryny same wieńczone są kodem 404.

Wobec zaistniałej sytuacji wykasuję problematyczną kolumnę.


#### Pobieranie danych

Porzebne będą:
1. Ilość emerytów ogółem (p2861, id 155055) dla lat 2009-2019 zagregowane dla województwa Mazowieckiego.
2. Stan ludności w wieku 45+ ogółem (p2137, id) dla lat 2009-2019 zagregowane dla regionu warszawskiego i mazowieckigo regionalnego.

#### Usuwanie kolumny

In [25]:
df_main_mod_noretires = pd.read_csv(DF_MAIN_MOD)

In [26]:
df_main_mod_noretires.drop('p2861-przeciętna-liczba-emerytów-i-rencistów-ogółem', axis=1, inplace=True)

### 4. Długości połączeń (p3571)

Dotyczą one ilości połączeń autobusowych (dalekobieżnych, miejskich, podmiejskich i regionalnych). Wybrakowana jest ponad połowa wpisów, a to przez to, że dane gromadzone są dopiero od roku 2014, dlatego uzupełnimy je średnimi.

In [27]:
nans_groupby_region = df_main_mod_noretires.drop(labels='ID_region', axis=1).isna().groupby(df_main_mod_noretires.ID_region)
lack = nans_groupby_region.sum().loc[:,['p3571-dalekobieżne', 'p3571-miejskie', 'p3571-podmiejskie', 'p3571-regionalne']]
overall = nans_groupby_region.count().loc[:,['p3571-dalekobieżne', 'p3571-miejskie', 'p3571-podmiejskie', 'p3571-regionalne']]

In [28]:
lack/overall

,p3571-dalekobieżne,p3571-miejskie,p3571-podmiejskie,p3571-regionalne
ID_region,,,,
11210000000,0.384977,0.384977,0.384977,0.384977
12410000000,0.396341,0.396341,0.396341,0.396341
20810000000,0.400000,0.400000,0.400000,0.400000
23010000000,0.400000,0.400000,0.400000,0.400000
23210000000,0.384236,0.384236,0.384236,0.384236
30210000000,0.387324,0.387324,0.387324,0.387324
31610000000,0.389831,0.389831,0.389831,0.389831
40410000000,0.408889,0.408889,0.408889,0.408889
42210000000,0.396985,0.396985,0.396985,0.396985


In [30]:
del nans_groupby_region
del lack
del overall

In [29]:
df_main_mod_noretires.set_index(['ID_region', 'ID_subregion', 'ID_powiatu', 'year']).loc[71410000000,['p3571-dalekobieżne', 'p3571-miejskie', 'p3571-podmiejskie', 'p3571-regionalne']][0:20]

p3571-dalekobieżne  p3571-miejskie  \
ID_subregion ID_powiatu  year                                       
71412800000  71412865000 2010                 NaN             NaN   
                         2011                 NaN             NaN   
                         2012                 NaN             NaN   
                         2013                 NaN             NaN   
                         2014                43.0             1.0   
                         2015                56.0             0.0   
                         2016                42.0             1.0   
                         2017                48.0             1.0   
                         2018                57.0             3.0   
                         2019                70.0             3.0   
71412900000  71412908000 2010                 NaN             NaN   
                         2011                 NaN             NaN   
                         2012                 NaN             NaN   
                         2013                 NaN             NaN   
                         2014                43.0             1.0   
                         2015                56.0             0.0   
                         2016                42.0             1.0   
                         2017                48.0             1.0   
                         2018                57.0             3.0   
                         2019                70.0             3.0   

                               p3571-podmiejskie  p3571-regionalne  
ID_subregion ID_powiatu  year                                       
71412800000  71412865000 2010                NaN               NaN  
                         2011                NaN               NaN  
                         2012                NaN               NaN  
                         2013                NaN               NaN  
                         2014              247.0              41.0  
                         2015              246.0              48.0  
                         2016              263.0              34.0  
                         2017              296.0              32.0  
                         2018              216.0              16.0  
                         2019              232.0              19.0  
71412900000  71412908000 2010                NaN               NaN  
                         2011                NaN               NaN  
                         2012                NaN               NaN  
                         2013                NaN               NaN  
                         2014              247.0              41.0  
                         2015              246.0              48.0  
                         2016              263.0              34.0  
                         2017              296.0              32.0  
                         2018              216.0              16.0  
                         2019              232.0              19.0

In [31]:
df_p3571_means = df_main_mod_noretires.groupby(['ID_region']).mean()[['p3571-dalekobieżne',
                                                                      'p3571-miejskie',
                                                                      'p3571-podmiejskie',
                                                                      'p3571-regionalne']]
df_main_mod_noretires = df_main_mod_noretires.set_index(['ID_region']).fillna(df_p3571_means)

del df_p3571_means

In [32]:
df_main_mod_noretires.loc[71410000000,['year','p3571-dalekobieżne', 'p3571-miejskie', 'p3571-podmiejskie', 'p3571-regionalne']][0:20]

,year,p3571-dalekobieżne,p3571-miejskie,p3571-podmiejskie,p3571-regionalne
ID_region,,,,,
71410000000,2010,52.666667,1.5,250.0,31.666667
71410000000,2011,52.666667,1.5,250.0,31.666667
71410000000,2012,52.666667,1.5,250.0,31.666667
71410000000,2013,52.666667,1.5,250.0,31.666667
71410000000,2014,43.000000,1.0,247.0,41.000000
71410000000,2015,56.000000,0.0,246.0,48.000000
71410000000,2016,42.000000,1.0,263.0,34.000000
71410000000,2017,48.000000,1.0,296.0,32.000000
71410000000,2018,57.000000,3.0,216.0,16.000000


### 5. Przewozy pasażerów (p3603)

Jak widać dla regionu warszawskiego i mazowieckiego brakuje tutaj danych do 2012 roku. Ponownie uzupełnimy średnią pamiętając, że poziomem agregacji tej zmiennej nadal są regiony.

In [34]:
df_main_mod_noretires.reset_index(inplace=True)
df_main_mod_noretires.set_index(['ID_region', 'ID_subregion', 'ID_powiatu', 'year']).loc[71410000000:, ['p3603-przewozy-pasażerskie-na-1-mieszkańca', 'p3603-przewozy-pasażerów-w-mln']][0:20]

p3603-przewozy-pasażerskie-na-1-mieszkańca  \
ID_region   ID_subregion ID_powiatu  year                                               
71410000000 71412800000  71412865000 2010                                         NaN   
                                     2011                                         NaN   
                                     2012                                         NaN   
                                     2013                                      290.05   
                                     2014                                      283.33   
                                     2015                                      288.77   
                                     2016                                      284.42   
                                     2017                                      287.15   
                                     2018                                      297.58   
                                     2019                                      311.29   
            71412900000  71412908000 2010                                         NaN   
                                     2011                                         NaN   
                                     2012                                         NaN   
                                     2013                                      290.05   
                                     2014                                      283.33   
                                     2015                                      288.77   
                                     2016                                      284.42   
                                     2017                                      287.15   
                                     2018                                      297.58   
                                     2019                                      311.29   

                                           p3603-przewozy-pasażerów-w-mln  
ID_region   ID_subregion ID_powiatu  year                                  
71410000000 71412800000  71412865000 2010                             NaN  
                                     2011                             NaN  
                                     2012                             NaN  
                                     2013                           850.6  
                                     2014                           837.3  
                                     2015                           859.7  
                                     2016                           852.8  
                                     2017                           866.7  
                                     2018                           905.4  
                                     2019                           955.7  
            71412900000  71412908000 2010                             NaN  
                                     2011                             NaN  
                                     2012                             NaN  
                                     2013                           850.6  
                                     2014                           837.3  
                                     2015                           859.7  
                                     2016                           852.8  
                                     2017                           866.7  
                                     2018                           905.4  
                                     2019                           955.7

In [35]:
df_p3603_means = df_main_mod_noretires.groupby(['ID_region']).mean()[['p3603-przewozy-pasażerskie-na-1-mieszkańca',
                                                                      'p3603-przewozy-pasażerów-w-mln']]
df_main_mod_noretires = df_main_mod_noretires.set_index(['ID_region']).fillna(df_p3603_means)

del df_p3603_means

In [36]:
df_main_mod_noretires.loc[71410000000,['p3603-przewozy-pasażerskie-na-1-mieszkańca', 'p3603-przewozy-pasażerów-w-mln']][0:20]

,p3603-przewozy-pasażerskie-na-1-mieszkańca,p3603-przewozy-pasażerów-w-mln
ID_region,,
71410000000,291.798571,875.457143
71410000000,291.798571,875.457143
71410000000,291.798571,875.457143
71410000000,290.050000,850.600000
71410000000,283.330000,837.300000
71410000000,288.770000,859.700000
71410000000,284.420000,852.800000
71410000000,287.150000,866.700000
71410000000,297.580000,905.400000


### 6. Loty (p1767)

Tranzyt i udział procentowy względem polski to zmienne silnie skorelowane z przylotami i wylotami, a dodatkowo są one wybitnie wybrakowane. Pozbycie się ich będzie najlepszym do obrania krokiem.<br>
Braki w odlotach i przylotach są jednoczesne, co sugeruje, że w danym regionie nie operuje żadne międzynarodowe lotnisko i w tym przypadku pozostawimy zera.

In [37]:
df_main_mod_noretires.reset_index(inplace=True)
df_main_mod_noretires.set_index(['ID_region', 'ID_subregion', 'ID_powiatu', 'year'], inplace=True)

In [39]:
df_main_mod_noretires.loc[df_main_mod_noretires['p1767-przyjazdy'].isna() |
                          df_main_mod_noretires['p1767-wyjazdy'].isna(),
                          ['p1767-przyjazdy', 'p1767-wyjazdy']]

p1767-przyjazdy  p1767-wyjazdy
ID_region   ID_subregion ID_powiatu  year                                
31610000000 31613100000  31613101000 2010              NaN            NaN
                                     2011              NaN            NaN
                                     2013              NaN            NaN
                                     2014              NaN            NaN
                                     2015              NaN            NaN
...                                                    ...            ...
71420000000 71427300000  71427338000 2011              NaN            NaN
                                     2012              NaN            NaN
                                     2013              NaN            NaN
                                     2014              NaN            NaN
                                     2019              NaN            NaN

[774 rows x 2 columns]

In [40]:
df_main_mod_noretires.loc[df_main_mod_noretires['p1767-przyjazdy'].isna() &
                          df_main_mod_noretires['p1767-wyjazdy'].isna(),
                          ['p1767-przyjazdy', 'p1767-wyjazdy']]

p1767-przyjazdy  p1767-wyjazdy
ID_region   ID_subregion ID_powiatu  year                                
31610000000 31613100000  31613101000 2010              NaN            NaN
                                     2011              NaN            NaN
                                     2013              NaN            NaN
                                     2014              NaN            NaN
                                     2015              NaN            NaN
...                                                    ...            ...
71420000000 71427300000  71427338000 2011              NaN            NaN
                                     2012              NaN            NaN
                                     2013              NaN            NaN
                                     2014              NaN            NaN
                                     2019              NaN            NaN

[774 rows x 2 columns]

In [41]:
df_main_mod_noretires.drop(['p1767-tranzyt', 'p1767-udział-ruchu-pasażerów-polska-100'], axis=1, inplace=True)

In [42]:
df_main_mod_noretires.fillna(0, inplace=True)

In [43]:
df_main_mod_noretires.to_csv(DF_MAIN_MOD_NORETIRES)

### Sprawdzenie

Sprawdzamy, czy w którejkolwiek kolumnie pozostała chociaż jedna nieokreślona wartość. False od góry do dołu świadczą, że wszystko przebiegło pomyślnie.

In [44]:
df_main_mod_noretires.isna().any()

p1767-przyjazdy                                                                                                                               False
p1767-wyjazdy                                                                                                                                 False
p2424-linie-kolejowe-ogółem-na-10-tysięcy-ludności                                                                                            False
p2424-linie-kolejowe-ogółem-na-100-km2                                                                                                        False
p2504-1-kwartał-ogółem                                                                                                                        False
p2504-2-kwartał-ogółem                                                                                                                        False
p2504-3-kwartał-ogółem                                                                                          